In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from datetime import datetime, timedelta

# Function to calculate Black-Scholes option price
def black_scholes_option_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")

    return option_price

# Example usage
def backtest_black_scholes(nifty_data, strike_price, expiration_date, risk_free_rate, volatility):
    # Assuming nifty_data is a pandas DataFrame with columns 'Date' and 'Close'
    nifty_data['Date'] = pd.to_datetime(nifty_data['Date'])
    end_date = pd.to_datetime(expiration_date)
    
    # Calculate time to expiration in years
    nifty_data['TimeToExpiration'] = (end_date - nifty_data['Date']) / timedelta(days=365)
    
    # Calculate Black-Scholes option prices for each date
    nifty_data['OptionPrice'] = nifty_data.apply(lambda row: black_scholes_option_price(row['Close'], strike_price, row['TimeToExpiration'], risk_free_rate, volatility), axis=1)
    
    return nifty_data[['Date', 'Close', 'TimeToExpiration', 'OptionPrice']]

# Example usage:
# Load historical Nifty50 data (replace with your actual data loading code)
nifty_data = pd.read_csv('historical_nifty50_data.csv')

# Set parameters
strike_price = 12000  # Example strike price
expiration_date = '2024-12-31'  # Example expiration date
risk_free_rate = 0.05  # Example risk-free rate (5%)
volatility = 0.2  # Example volatility (20%)

# Backtest Black-Scholes model
backtest_result = backtest_black_scholes(nifty_data, strike_price, expiration_date, risk_free_rate, volatility)

# Print or visualize the backtest results
print(backtest_result)
